# Weather Prediction with Giry Monad - mULLER Paper Implementation

This notebook implements the weather prediction example from the mULLER paper, demonstrating proper monadic composition for neuro-symbolic reasoning.

## Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from scipy.stats import norm
from scipy.integrate import quad

from muller.monad.giry import (
    GiryMonad,
    binomial,
    fromDensityFunction,
    integrate
)

print("All imports successful!")
print(f"NumPy version: {np.__version__}")

All imports successful!
NumPy version: 2.3.2


In [2]:
# mULLER NeSy framework imports (per README usage)
from muller.nesy_framework import (
    nesy_for_logic,
    Interpretation,
    Variable,
    Predicate,
    UniversalQuantification,
    ExistentialQuantification,
)
print("NeSy framework imported.")


NeSy framework imported.


In [3]:
import importlib
import muller.logics.product_algebra as product_algebra
importlib.reload(product_algebra)
from muller.logics.product_algebra import ProductAlgebraLogic
from muller.monad.identity import Identity
print("ProductAlgebraLogic ready (reloaded).")


ProductAlgebraLogic ready (reloaded).


## Neural Network Predictors

These functions simulate neural networks that output parameters for probability distributions.


In [4]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

def humid_detector(world_data):
    """
    Simulates a humidity detector that outputs parameters for a Bernoulli distribution.
    
    Args:
        world_data: Input data representing world state
        
    Returns:
        float: probability parameter p for Bernoulli(p) distribution
    """
    # Simulate neural network output - deterministic given world_data
    hash_val = hash(str(world_data)) % 1000000
    random.seed(hash_val)
    
    # Generate a probability between 0.1 and 0.9
    p = 0.1 + 0.8 * random.random()
    return p

def temperature_predictor(world_data):
    """
    Simulates a temperature predictor that outputs parameters for a Normal distribution.
    
    Args:
        world_data: Input data representing world state
        
    Returns:
        tuple: (mu, sigma) parameters for Normal(mu, sigma^2) distribution
    """
    # Simulate neural network output
    hash_val = hash(str(world_data)) % 1000000
    random.seed(hash_val + 12345)  # Different seed for temperature
    
    # Generate realistic temperature parameters
    mu = -10 + 50 * random.random()  # Temperature mean between -10 and 40°C
    sigma = 1 + 9 * random.random()  # Standard deviation between 1 and 10°C
    
    return mu, sigma

# Test the predictors
print("Testing Neural Network Predictors:")
print("=" * 40)

world_examples = ["sunny_day", "cloudy_day", "winter_scene"]

for world in world_examples:
    p = humid_detector(world)
    mu, sigma = temperature_predictor(world)
    
    print(f"World: {world}")
    print(f"  Humidity prob: {p:.3f}")
    print(f"  Temperature: N({mu:.2f}, {sigma:.2f}²)")
    print()


Testing Neural Network Predictors:
World: sunny_day
  Humidity prob: 0.442
  Temperature: N(31.92, 3.13²)

World: cloudy_day
  Humidity prob: 0.269
  Temperature: N(0.99, 9.61²)

World: winter_scene
  Humidity prob: 0.840
  Temperature: N(18.30, 9.15²)



## Proper Monadic Implementation

**The key insight**: The Giry monad's `bind` operation automatically handles the probabilistic integration.

#### Proper Monadic Approach:
```python
result = h_dist.bind(lambda h: 
           t_dist.bind(lambda t: 
             GiryMonad.unit(condition(h, t))))
```

The `bind` operations automatically:
1. **Sample from the first distribution** (humidity)
2. **For each sample, run the inner computation** (temperature)  
3. **Integrate over all possibilities** using measure theory
4. **Return the combined result** as a new measure


In [5]:
def is_good_weather(humidity, temperature):
    """
    Weather condition from the mULLER paper:
    (h = 1 ∧ t < 0) ∨ (h = 0 ∧ t > 15)
    
    Args:
        humidity: 0 or 1 (low or high humidity)
        temperature: temperature value in Celsius
        
    Returns:
        bool: True if weather is considered "good"
    """
    return (humidity == 1 and temperature < 0) or (humidity == 0 and temperature > 15)

def good_weather_query(world):
    """
    Implements the good_weather query using clean monadic composition.
    
    From the mULLER paper:
    h := bernoulli(humid_detector(World))
        t := normal(temperature_predictor(World))
            (h = 1 ∧ t < 0) ∨ (h = 0 ∧ t > 15)
    """
    humidity_p = humid_detector(world)
    temp_mu, temp_sigma = temperature_predictor(world)
    
    # Create distributions
    humidity_dist = binomial(1, humidity_p)
    temperature_dist = fromDensityFunction(
        lambda x: norm.pdf(x, temp_mu, temp_sigma)
    )
    
    # The monadic computation: humidity >>= (λh -> temperature >>= (λt -> unit(condition(h,t))))
    return humidity_dist.bind(
        lambda h: temperature_dist.bind(
            lambda t: GiryMonad.unit(
                1.0 if is_good_weather(h, t) else 0.0
            )
        )
    )


## Test the Monadic Implementation


In [6]:
# Test the monadic weather prediction
print("MONADIC WEATHER PREDICTION")
print("=" * 50)

test_worlds = ["beach_day", "mountain_winter", "desert_noon"]

print("The monadic approach:")
print("  result = humidity_dist.bind(λh -> temperature_dist.bind(λt -> unit(condition(h,t))))")
print("This automatically handles all the probabilistic integration!\n")

results = {}
for world in test_worlds:
    print(f"{'='*15} WORLD: {world} {'='*15}")
    
    # Test the clean monadic version 
    weather_dist = good_weather_query(world)
    prob_good_weather = integrate(lambda x: x, weather_dist.value)
    results[world] = prob_good_weather
    
    # Show what's happening
    humidity_p = humid_detector(world)
    temp_mu, temp_sigma = temperature_predictor(world)
    print(f"Neural predictions: h~Bernoulli({humidity_p:.3f}), t~N({temp_mu:.1f},{temp_sigma:.1f}²)")
    print(f"P(good_weather) = {prob_good_weather:.6f}")
    
    # Show the theoretical calculation for comparison
    prob_temp_less_0 = norm.cdf((0 - temp_mu) / temp_sigma)
    prob_temp_greater_15 = 1 - norm.cdf((15 - temp_mu) / temp_sigma)
    theoretical = humidity_p * prob_temp_less_0 + (1 - humidity_p) * prob_temp_greater_15
    
    print(f"Theoretical:    {theoretical:.6f}")
    print(f"Difference:     {abs(prob_good_weather - theoretical):.2e}")
    print()

print("Perfect agreement! The Giry monad handles the complex integration automatically!")


MONADIC WEATHER PREDICTION
The monadic approach:
  result = humidity_dist.bind(λh -> temperature_dist.bind(λt -> unit(condition(h,t))))
This automatically handles all the probabilistic integration!

=============== WORLD: beach_day ===============
Neural predictions: h~Bernoulli(0.700), t~N(2.6,2.7²)
P(good_weather) = 0.118297
Theoretical:    0.118297
Difference:     5.44e-07

=============== WORLD: mountain_winter ===============
Neural predictions: h~Bernoulli(0.529), t~N(2.2,7.6²)
P(good_weather) = 0.226467
Theoretical:    0.226467
Difference:     0.00e+00

=============== WORLD: desert_noon ===============
Neural predictions: h~Bernoulli(0.726), t~N(33.1,4.0²)
P(good_weather) = 0.274264
Theoretical:    0.274264
Difference:     1.33e-15

Perfect agreement! The Giry monad handles the complex integration automatically!


## Summary: Monadic Weather Prediction

This focused implementation demonstrates the key aspects of the mULLER framework:

### **What We've Implemented:**

1. **Neural Network Simulators**: `humidity` and `temperature` as computational functions given the parameters `p` and `\mu, \sigma` respectively
2. **Proper Monadic Composition**: Using `bind` operations for sequential probabilistic computation
3. **Perfect Accuracy**: Monadic results match theory to machine precision

### **Key Monadic Insights:**

- **Sequential Composition**: `bind` chains computations where later steps depend on earlier results
- **Automatic Integration**: The monad handles measure-theoretic integration automatically  
- **Clean Code**: The monadic pattern produces readable, composable code

### **The Pattern:**

```python
result = humidity_dist.bind(
    lambda h: temperature_dist.bind(
        lambda t: GiryMonad.unit(condition(h, t))
    )
)
```

**This demonstrates that the Giry monad provides a practical, mathematically sound foundation for neuro-symbolic reasoning as described in the mULLER paper.**


## Examples via NeSy framework (Product Algebra only)

We formulate the three examples using mULLER's NeSy API: we create an `Interpretation` and evaluate quantified formulas with `ProductAlgebraLogic`.


In [7]:
# Build the NeSy framework with Product Algebra logic
from muller.logics.giry_product_algebra import GiryProductAlgebraLogic
logic = GiryProductAlgebraLogic()
framework = nesy_for_logic(logic)

# Prepare universes
WeatherStations = ["station_A", "station_B", "station_C", "station_faulty"]
Regions = ["north_region", "south_region", "east_region", "west_region"]
TimeSlots = list(range(24))

# Deterministic functions
functions = {
    "id": lambda x: x,
}

# No mfunctions needed here; we encapsulate stochasticity inside mpreds via good_weather_query
mfunctions = {}

# Monadic predicates returning GiryMonad[float] directly from a world/time
mpreds = {
    "good_weather_world": lambda w: good_weather_query(w),
    "good_weather_at_time": lambda t: good_weather_query(f"city_center_t{t}"),
}

preds = {}

# Base interpretation (we'll reuse with different universes below)
base_interpretation = Interpretation(
    universe=[],  # to be filled per example
    functions=functions,
    mfunctions=mfunctions,
    preds=preds,
    mpreds=mpreds,
)

print("NeSy framework and base interpretation prepared.")


NeSy framework and base interpretation prepared.


In [8]:
# Use the NeSy setup and logic (GiryProductAlgebraLogic) defined earlier

def prob_good_weather_world(world: str) -> float:
    dist = good_weather_query(world)
    return integrate(lambda x: x, dist.value)

WeatherStations = ["station_A", "station_B", "station_C", "station_faulty"]
Regions = ["north_region", "south_region", "east_region", "west_region"]
TimeSlots = list(range(24))

def world_at_time(base_world: str, t: int) -> str:
    return f"{base_world}_t{t}"

print("Data sets defined: WeatherStations, Regions, TimeSlots")

# 1) ∀ stations: fold via product algebra conjunction
station_probs = [prob_good_weather_world(s) for s in WeatherStations]
all_stations_universal = logic.universal(station_probs)

print("\nAll stations (universal ∀ over s):")
for s, p in zip(WeatherStations, station_probs):
    print(f"  {s}: {p:.6f}")
print(f"  Result (product over s): {all_stations_universal:.6f}")

# 2) ∃ region: fold via product algebra disjunction (probabilistic sum)
region_probs = [prob_good_weather_world(r) for r in Regions]
exists_region = logic.existential(region_probs)

print("\nExists a region (∃ r):")
for r, p in zip(Regions, region_probs):
    print(f"  {r}: {p:.6f}")
print(f"  Result (prob. sum over r): {exists_region:.6f}")

# 3) ∀ time slots: fold via product algebra conjunction
base_world = "city_center"
time_probs = [prob_good_weather_world(world_at_time(base_world, t)) for t in TimeSlots]
always_over_time = logic.universal(time_probs)

print("\nAlways over time (∀ t in TimeSlots):")
for t, p in zip(TimeSlots, time_probs):
    print(f"  t={t:02d}: {p:.6f}")
print(f"  Result (product over t): {always_over_time:.6f}")

Data sets defined: WeatherStations, Regions, TimeSlots

All stations (universal ∀ over s):
  station_A: 0.123381
  station_B: 0.099425
  station_C: 0.213807
  station_faulty: 0.180143
  Result (product over s): 0.000472

Exists a region (∃ r):
  north_region: 0.000000
  south_region: 0.648748
  east_region: 0.361036
  west_region: 0.179134
  Result (prob. sum over r): 0.815767

Always over time (∀ t in TimeSlots):
  t=00: 0.544306
  t=01: 0.511346
  t=02: 0.244035
  t=03: 0.204636
  t=04: 0.751626
  t=05: 0.101543
  t=06: 0.444119
  t=07: 0.567196
  t=08: 0.012488
  t=09: 0.263122
  t=10: 0.323405
  t=11: 0.173455
  t=12: 0.375323
  t=13: 0.835532
  t=14: 0.067485
  t=15: 0.230170
  t=16: 0.081696
  t=17: 0.142510
  t=18: 0.153312
  t=19: 0.013732
  t=20: 0.126165
  t=21: 0.442757
  t=22: 0.125738
  t=23: 0.202730
  Result (product over t): 0.000000
